In [1]:
include("output.jl")
using ImpvolOutput, FileIO
results = load("experiments/baseline/actual/results.jld2")["results"]
parameters = load("experiments/baseline/common_parameters.jld2")["parameters"]
real_GDP_model = sum(ImpvolOutput.make_series(results, :real_GDP), 3)

1×25×1×36 Array{Float64,4}:
[:, :, 1, 1] =
 63855.7  31864.1  38824.1  95853.6  …  37377.2  2.04764e5  1.13267e6

[:, :, 1, 2] =
 66268.0  31200.5  41837.8  1.04521e5  …  38398.4  2.13385e5  1.20157e6

[:, :, 1, 3] =
 63178.7  32255.5  41129.6  1.11434e5  …  40167.5  2.07574e5  1.18479e6

...

[:, :, 1, 34] =
 1.58892e5  69357.0  86544.0  268267.0  …  74405.3  4.81181e5  3.17756e6

[:, :, 1, 35] =
 1.7897e5  77431.3  90752.4  2.99321e5  …  83274.9  5.32559e5  3.46775e6

[:, :, 1, 36] =
 2.04374e5  88861.0  1.04261e5  3.6746e5  …  94962.5  6.19506e5  3.89584e6

In [2]:
rows = [("1970s", 1:8), ("1980s", 9:18), ("1990s", 19:28), ("2000s", 28:36)]
columns = ["actual", "kappa1972", "nosectoral", "nosectoral_kappa1972"]

4-element Array{String,1}:
 "actual"              
 "kappa1972"           
 "nosectoral"          
 "nosectoral_kappa1972"

In [3]:
function get_volatility(rows, column)
    results = load("experiments/baseline/$(column)/results.jld2")["results"]
    real_GDP_model = sum(ImpvolOutput.make_series(results, :real_GDP), 3)
    return [mean(ImpvolOutput.calculate_volatilities(real_GDP_model, parameters, true, row[2])) for row in rows]
end

get_volatility (generic function with 1 method)

In [4]:
using DataFrames
volatility = DataFrame()
volatility[:decades] = [row[1] for row in rows]
for column in columns
    volatility[Symbol(column)] = get_volatility(rows, column)
end


In [5]:
volatility

,decades,actual,kappa1972,nosectoral,nosectoral_kappa1972
1,1970s,0.000844001,0.000881151,0.00112046,0.00123903
2,1980s,0.00103177,0.00117726,0.00117976,0.00156798
3,1990s,0.00101973,0.00154694,0.00157059,0.0024906
4,2000s,0.00099393,0.00179611,0.00133817,0.00226184


In [6]:
stats = volatility
stats[:trade_barriers] = 100 * (stats[:actual] - stats[:kappa1972]) ./ stats[:kappa1972]
stats[:diversification] = 100 * (stats[:nosectoral] - stats[:nosectoral_kappa1972]) ./ stats[:kappa1972]
stats[:specialization] = 100 * (stats[:actual] - stats[:kappa1972] - stats[:nosectoral] + stats[:nosectoral_kappa1972]) ./ stats[:kappa1972]


4-element Array{Float64,1}:
  9.23955
 20.6189 
 25.3917 
  6.76371

In [7]:
for col in columns
    delete!(volatility, Symbol(col))
end

In [8]:
using CSV
CSV.write("volatility_by_decade.csv", volatility)

CSV.Sink{Void,DataType}(    CSV.Options:
        delim: ','
        quotechar: '"'
        escapechar: '\\'
        missingstring: ""
        dateformat: nothing
        decimal: '.'
        truestring: 'true'
        falsestring: 'false'
        internstrings: true, IOBuffer(data=UInt8[...], readable=true, writable=true, seekable=true, append=false, size=0, maxsize=Inf, ptr=1, mark=-1), "volatility_by_decade.csv", 54, true, String["decades", "trade_barriers", "diversification", "specialization"], 4, false, Val{false})